In [ ]:
import matplotlib.pyplot as pl
pl.style.use('apw-notebook')
%matplotlib inline

In [ ]:
import astropy.units as u
import numpy as np
from pygaia.errors.astrometric import parallaxErrorSkyAvg, properMotionErrorSkyAvg
from pygaia.photometry.transformations import gminvFromVmini

from gary.observation import distance, distance_modulus

In [ ]:
n_samples = 16384
D = np.logspace(-1, 3, 256)*u.kpc # 5*u.kpc
plx = D.to(u.microarcsecond, equivalencies=u.parallax())
DM = distance_modulus(D)

In [ ]:
def fractional_scatter(M_V, scatter):
    dm = distance_modulus(D)
    m = dm + M_V
    
    m_scatter = np.random.normal(m, scatter, size=(n_samples, D.size))
    dm_scatter = m_scatter - M_V
    sigma_D = np.std(distance(dm_scatter).to(u.kpc), axis=0)
    
    return m,sigma_D,np.median((sigma_D / D).value)

In [ ]:
def parallax_plot(m_G, V_I):
    sigma_plx = parallaxErrorSkyAvg(m_G, V_I) * u.microarcsecond
    pl.loglog(D, sigma_plx / plx, ls='-', lw=2, marker=None)

    pl.xlabel("$d$ [kpc]")
    pl.ylabel(r"$\sigma_\pi / \pi$")

In [ ]:
names = ["name", "M_V", "V-I", "M_G", "phot_err"]
all_types = []

## MSTO

In [ ]:
M_V = 4.
V_I = 0.50

M_G = gminvFromVmini(V_I) + M_V
m_G = DM + M_G

In [ ]:
scatter = 0.9
m,sig_D,frac_scatter_single = fractional_scatter(M_V, scatter)
frac_scatter_single

In [ ]:
all_types.append(['MSTO', M_V, V_I, M_G, frac_scatter_single])

A population of MSTO stars

In [ ]:
scatter = 0.2
m,sig_D,frac_scatter_many = fractional_scatter(M_V, scatter)
frac_scatter_many

Parallax errors

In [ ]:
parallax_plot(m_G, V_I)
pl.axhline(frac_scatter_single)
pl.axhline(frac_scatter_many, linestyle='--')

## BHB

In [ ]:
M_V = 0.5
V_I = -0.16
scatter = 0.15

M_G = gminvFromVmini(V_I) + M_V
m_G = distance_modulus(D) + M_G

m,sig_D,frac_scatter = fractional_scatter(M_V, scatter)
print(frac_scatter)

all_types.append(['BHB', M_V, V_I, M_G, frac_scatter])

In [ ]:
parallax_plot(m_G, V_I)
pl.axhline(frac_scatter)

## RRL

In [ ]:
M_V = 0.5
V_I = 0.24
scatter = 0.15

M_G = gminvFromVmini(V_I) + M_V
m_G = distance_modulus(D) + M_G

m,sig_D,frac_scatter = fractional_scatter(M_V, scatter)
print(frac_scatter)

all_types.append(['RRL', M_V, V_I, M_G, 0.02])

In [ ]:
parallax_plot(m_G, V_I)
pl.axhline(frac_scatter)
pl.axhline(0.02, ls='--')

## Red clump

http://arxiv.org/abs/astro-ph/0208057

In [ ]:
M_V = 0.7 
V_I = 0.8
scatter = 0.2

M_G = gminvFromVmini(V_I) + M_V
m_G = distance_modulus(D) + M_G

m,sig_D,frac_scatter = fractional_scatter(M_V, scatter)
print(frac_scatter)

all_types.append(['RC', M_V, V_I, M_G, frac_scatter])

In [ ]:
parallax_plot(m_G, V_I)
pl.axhline(frac_scatter)

## K giant

In [ ]:
M_V = -1. # 1 to -3
V_I = 0.75
scatter = 0.4 # Xue, X. et al. 2014

M_G = gminvFromVmini(V_I) + M_V
m_G = distance_modulus(D) + M_G

m,sig_D,frac_scatter = fractional_scatter(M_V, scatter)
print(frac_scatter)

all_types.append(['KG', M_V, V_I, M_G, frac_scatter])

In [ ]:
parallax_plot(m_G, V_I)
pl.axhline(frac_scatter)

## M giant

https://ui.adsabs.harvard.edu/#abs/2000AJ....120.2550M/abstract

In [ ]:
M_V = -1. # 1 to -3
V_I = 1.67
scatter = 0.45

M_G = gminvFromVmini(V_I) + M_V
m_G = distance_modulus(D) + M_G

m,sig_D,frac_scatter = fractional_scatter(M_V, scatter)
print(frac_scatter)

all_types.append(['MG', M_V, V_I, M_G, frac_scatter])

In [ ]:
parallax_plot(m_G, V_I)
pl.axhline(frac_scatter)

---

## All together now

In [ ]:
from astropy.table import Table

In [ ]:
dtype = [str] + [float]*(len(names)-1)
tbl = Table(np.array(all_types), names=names, dtype=dtype)

In [ ]:
colors = ["#a6d96a", "#542788", "#4393c3", "#8c510a", "#e08214", "#d73027"]
styles = ['-', '--', '-', '-.', ':', '--']

xlim = (1,250) # kpc
xticks = [1, 10, 100, 250]

### Absolute parallax error

In [ ]:
fig,ax = pl.subplots(1,1,figsize=(5.2,5.2))

for row,c,s in zip(tbl, colors, styles):
    m_G = DM + row['M_G']
    sigma_plx = parallaxErrorSkyAvg(m_G, row['V-I']) * u.microarcsecond
    
    err = sigma_plx
#     err[err > row['phot_err']] = row['phot_err']
    
    ax.loglog(D, err.value, color=c, ls=s, lw=3, marker=None, label=row['name'])
    
ax.set_xlabel("$d$ [kpc]")
ax.set_ylabel(r"$\sigma_\pi$ [$\mu$as]")
ax.set_ylim(1e0, 1e6)

ax.set_xlim(xlim)
ax.set_xticks(xticks)
ax.set_xticklabels([str(x) for x in xticks])

# ax.set_yticks([1e-2,1e-1,1e0])
# ax.set_yticklabels(["1%","10%","100%"])

ax.legend(loc='upper left', ncol=3)

fig.tight_layout()
fig.savefig("../figures/plx-err.pdf")
fig.savefig("../figures/plx-err.png", dpi=400)

### Fractional distance / parallax error

In [ ]:
fig,ax = pl.subplots(1,1,figsize=(5.2,5.2))

for row,c,s in zip(tbl, colors, styles):
    m_G = DM + row['M_G']
    sigma_plx = parallaxErrorSkyAvg(m_G, row['V-I']) * u.microarcsecond
    
    err = sigma_plx / plx
    err[err > row['phot_err']] = row['phot_err']
    
    ax.loglog(D, err, color=c, ls=s, lw=3, marker=None, label=row['name'])
    
ax.set_xlabel("$d$ [kpc]")
ax.set_ylabel(r"$\sigma_d / d$")
ax.set_ylim(1e-2, 1.)

ax.set_xlim(xlim)
ax.set_xticks(xticks)
ax.set_xticklabels([str(x) for x in xticks])

ax.set_yticks([1e-2,1e-1,1e0])
ax.set_yticklabels(["1%","10%","100%"])

# ax.legend(loc='upper left', ncol=3)

fig.tight_layout()
fig.savefig("../figures/frac-plx-err.pdf")
fig.savefig("../figures/frac-plx-err.png", dpi=400)

### Proper motion error

In [ ]:
fig,ax = pl.subplots(1,1,figsize=(5.2,5.2))

for row,c,s in zip(tbl, colors, styles):
    m_G = DM + row['M_G']
    
    # now velocity stuff
    sigma_pm = properMotionErrorSkyAvg(m_G, row['V-I']) * u.microarcsecond/u.yr    
    sigma_pm = np.sqrt(np.sum(sigma_pm**2, axis=0))
    ax.loglog(D, sigma_pm.value, color=c, ls=s, lw=3, marker=None, label=row['name'])
    
ax.set_xlabel("$d$ [kpc]")
ax.set_ylabel(r"$\sigma_{\rm \mu}$ [$\mu$as/yr]")
ax.set_ylim(1, 1e6)

ax.set_xlim(xlim)
ax.set_xticks(xticks)
ax.set_xticklabels([str(x) for x in xticks])

# ax.set_yticks([1,10,100])
# ax.set_yticklabels(["1","10","100"])

# ax.legend(loc='upper left')

fig.tight_layout()

fig.savefig("../figures/pm-err.pdf")
fig.savefig("../figures/pm-err.png", dpi=400)

### Tangential velocity error

In [ ]:
assumed_v = 100. * u.km/u.s
assumed_pm = (assumed_v / D).to(u.microarcsecond/u.yr, equivalencies=u.dimensionless_angles())

fig,ax = pl.subplots(1,1,figsize=(5.2,5.2))

for row,c,s in zip(tbl, colors, styles):
    m_G = DM + row['M_G']
    sigma_plx = parallaxErrorSkyAvg(m_G, row['V-I']) * u.microarcsecond
    err = sigma_plx / plx
    err[err > row['phot_err']] = row['phot_err']
    
    # now velocity stuff
    sigma_pm = properMotionErrorSkyAvg(m_G, row['V-I']) * u.microarcsecond/u.yr
    sigma_pm = np.sqrt(np.sum(sigma_pm**2, axis=0))
    
    term1 = (D**2 * sigma_pm**2).to(u.km**2/u.s**2, equivalencies=u.dimensionless_angles())
    term2 = ((D*err)**2 * assumed_pm**2).to(u.km**2/u.s**2, equivalencies=u.dimensionless_angles())
    
    vtan_err = np.sqrt(term1 + term2)
    ax.loglog(D, vtan_err, color=c, ls=s, lw=3, marker=None, label=row['name'])
    
ax.set_xlabel("$d$ [kpc]")
ax.set_ylabel(r"$\sigma_{\rm v_{tan}}$ [km/s]")
ax.set_ylim(1, 100.)

ax.set_xlim(xlim)
ax.set_xticks(xticks)
ax.set_xticklabels([str(x) for x in xticks])

ax.set_yticks([1,10,100])
ax.set_yticklabels(["1","10","100"])

# ax.legend(loc='upper left')

fig.tight_layout()
fig.savefig("../figures/vtan-err.pdf")
fig.savefig("../figures/vtan-err.png", dpi=400)